### Build a Question Answering application Using Graph Database

In [56]:
import os
from dotenv import load_dotenv
load_dotenv()

neo4j_uri = os.getenv("NEO4J_URI")
neo4j_password = os.getenv("NEO4J_PASSWORD")
neo4j_username = os.getenv("NEO4J_USERNAME")
groq_api_key = os.getenv("GROQ_API_KEY")

In [57]:
from langchain_community.graphs import Neo4jGraph

graph = Neo4jGraph(url=neo4j_uri, username=neo4j_username, password=neo4j_password)

In [58]:
# Dataset 
moview_query="""
LOAD CSV WITH HEADERS FROM
'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv' as row

MERGE(m:Movie{id:row.movieId})
SET m.released = date(row.released),
    m.title = row.title,
    m.imdbRating = toFloat(row.imdbRating)
FOREACH (director in split(row.director, '|') | 
    MERGE (p:Person {name:trim(director)})
    MERGE (p)-[:DIRECTED]->(m))
FOREACH (actor in split(row.actors, '|') | 
    MERGE (p:Person {name:trim(actor)})
    MERGE (p)-[:ACTED_IN]->(m))
FOREACH (genre in split(row.genres, '|') | 
    MERGE (g:Genre {name:trim(genre)})
    MERGE (m)-[:IN_GENRE]->(g))


"""

In [59]:
# graph.query(moview_query)

In [60]:
graph.refresh_schema()
print(graph.schema)

Node properties:
User {name: STRING, age: INTEGER, city: STRING, userId: INTEGER}
Post {postId: INTEGER, content: STRING, timestamp: DATE_TIME}
Movie {id: STRING, released: DATE, title: STRING, imdbRating: FLOAT}
Person {name: STRING}
Genre {name: STRING}
Relationship properties:

The relationships:
(:User)-[:POSTED]->(:Post)
(:User)-[:LIKES]->(:User)
(:User)-[:FRIEND]->(:User)
(:Movie)-[:IN_GENRE]->(:Genre)
(:Person)-[:DIRECTED]->(:Movie)
(:Person)-[:ACTED_IN]->(:Movie)


In [61]:
examples = [
    {
        "question": "How many artists are there?",
        "query": "MATCH (a:Person)-[:ACTED_IN]->(:Movie) RETURN count(DISTINCT a)",
    },
    {
        "question": "Which actors played in the movie Casino?",
        "query": "MATCH (m:Movie {{title: 'Casino'}})<-[:ACTED_IN]-(a) RETURN a.name",
    },
    {
        "question": "How many movies has Tom Hanks acted in?",
        "query": "MATCH (a:Person {name: 'Tom Hanks'})-[:ACTED_IN]->(m:Movie) RETURN count(m)",
    },
    {
        "question": "List all the genres of the movie Schindler's List",
        "query": "MATCH (m:Movie {{title: 'Schindler\\'s List'}})-[:IN_GENRE]->(g:Genre) RETURN g.name",
    },
    {
        "question": "Which actors have worked in movies from both the comedy and action genres?",
        "query": "MATCH (a:Person)-[:ACTED_IN]->(:Movie)-[:IN_GENRE]->(g1:Genre), (a)-[:ACTED_IN]->(:Movie)-[:IN_GENRE]->(g2:Genre) WHERE g1.name = 'Comedy' AND g2.name = 'Action' RETURN DISTINCT a.name",
    },
    {
        "question": "Which directors have made movies with at least three different actors named 'John'?",
        "query": "MATCH (d:Person)-[:DIRECTED]->(m:Movie)<-[:ACTED_IN]-(a:Person) WHERE a.name STARTS WITH 'John' WITH d, COUNT(DISTINCT a) AS JohnsCount WHERE JohnsCount >= 3 RETURN d.name",
    },
    {
        "question": "Identify movies where directors also played a role in the film.",
        "query": "MATCH (p:Person)-[:DIRECTED]->(m:Movie), (p)-[:ACTED_IN]->(m) RETURN m.title, p.name",
    },
    {
        "question": "Find the actor with the highest number of movies in the database.",
        "query": "MATCH (a:Actor)-[:ACTED_IN]->(m:Movie) RETURN a.name, COUNT(m) AS movieCount ORDER BY movieCount DESC LIMIT 1",
    },
]

In [62]:
from langchain_core.prompts import FewShotPromptTemplate,PromptTemplate

example_prompt=PromptTemplate.from_template(
    "User input:{question}\n Cypher query:{query}"
)

prompt=FewShotPromptTemplate(
    examples=examples[:5],
    example_prompt=example_prompt,
    prefix="You are a Neo4j expert. Given an input question,create a syntactically very accurate Cypher query",
    suffix="User input: {question}\nCypher query: ",
    input_variables=["question","schema"]
)

In [65]:
try:
    result = prompt.format(question="How many artists are there?", schema="foo")
    print(result)
except KeyError as e:
    print(f"KeyError occurred: {e}")
    print("Available keys:", prompt.input_variables)

KeyError occurred: 'name'
Available keys: ['question']


In [37]:
from langchain_groq import ChatGroq

llm = ChatGroq(groq_api_key=groq_api_key, model="gemma2-9b-it")

In [43]:
from langchain.chains import GraphCypherQAChain
chain=GraphCypherQAChain.from_llm(graph=graph,llm=llm,cypher_prompt=prompt,verbose=True)

In [54]:
response=chain.invoke("Which actors played in the movie Casino?")
response



> Entering new GraphCypherQAChain chain...


KeyError: 'name'

In [40]:
response=chain.invoke({"query":"How many persons are in Jumanji?"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie {title: 'Jumanji'})<-[:DIRECTED]-(p:Person)
RETURN count(p)
Full Context:
[{'count(p)': 1}]

> Finished chain.


{'query': 'How many persons are in Jumanji?',
 'result': "I don't know the answer. \n"}